run GraphcastOperational with 1 GPU and 80 GB memory

Install uv, if necessary. Initialize earth2studio-mpas project ahijevyc/mpas branch and extra graphcast dependency.
Add kernel for Jupyterhub

 ```
curl -LsSf https://astral.sh/uv/install.sh | sh
mkdir earth2studio-mpas
cd earth2studio-mpas
uv init
uv add "earth2studio[graphcast] @ git+https://github.com/ahijevyc/earth2studio.git@ahijevyc/mpas"
uv add metpy
uv add --dev ipykernel ipywidgets
uv run python -m ipykernel install --user --name=earth2studio-mpas --display-name "Python (Earth2Studio MPAS)"
```

In [ ]:
import os
from pathlib import Path

import pandas as pd
import xarray as xr
from earth2studio.data.mpas import MPASHybrid
from earth2studio.io import NetCDF4Backend, WPSBackend
from earth2studio.models.px import GraphCastOperational
from earth2studio.run import deterministic

SCRATCH = Path(os.getenv("SCRATCH"))
ai_models_dir = SCRATCH / "ai-models"
date = pd.to_datetime("2018042400", format="%Y%m%d%H")

In [ ]:
!nvidia-smi

In [ ]:
if True:
    idir = Path(
        "/glade/campaign/mmm/parc/stoedtli/hourly_cycling/6h_cycling/stoedtli_3dhybrid-60-60-iter_O30kmI60km_benchmark_1/CyclingFC"
    )
    mpas_datasrc = MPASHybrid(
        grid_path=idir / f"{date:%Y%m%d%H}/invariant.655362.nc",
        data_path=f"{idir}/%Y%m%d%H/mpasout.%Y-%m-%d_%H.%M.%S.nc",
        pressure_levels=[50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000],
    )

if False:
    idir = Path("/glade/campaign/mmm/parc/schwartz")
    mpas_datasrc = MPASEns(
        grid_path=idir / "MPAS/15-3km_mesh/grid_mesh/x5.6488066.grid_CONUS.nc",
        data_dir=idir / f"HWT{date.year}/mpas",
    )
if False:
    mpas_datasrc = MPASEns(
        grid_path=idir / "MPAS/15km_mesh/grid_mesh/x1.2621442.grid.nc",
        data_dir=idir / f"HWT{date.year}/mpas_15km",
    )

if False:
    idir = Path(
        "/glade/campaign/mmm/parc/stoedtli/hourly_cycling/6h_cycling/stoedtli_3dhybrid-60-60-iter_O30kmI60km_benchmark_1/CyclingFC"
    )
    mpas_datasrc = MPASPres(
        grid_path=idir / f"{date:%Y%m%d%H}/invariant.655362.nc",
        data_path=[
            f"{idir}/%Y%m%d%H/diag.%Y-%m-%d_%H.%M.%S.nc",
            f"{idir}/%Y%m%d%H/p_sfc.%Y-%m-%d_%H.%M.%S.nc",
        ],
    )

In [ ]:
model_class = GraphCastOperational
model = model_class.load_model(model_class.load_default_package())

In [ ]:
ds = mpas_datasrc(
    ["20180423T18"],
    ["z1000", "z500", "z", "lsm"],
)
ds

In [ ]:
ds.sel(variable="msl").plot(col="time", robust=False, cmap="nipy_spectral")

In [ ]:
model.input_coords()["variable"]

In [ ]:
model_abbrev = model.__class__.__name__.upper()[0:4]

nsteps = 1
ofile = SCRATCH / f"tmp/test_{model_abbrev}:{date:%Y-%m-%d_%H}.nc"
if os.path.exists(ofile):
    os.remove(ofile)
io = WPSBackend(SCRATCH / "tmp", model_source=model.__class__.__name__, static_fields=["lsm", "z"])
# io = NetCDF4Backend(ofile, backend_kwargs={"mode": "w"})
io = deterministic([date], nsteps, model, mpas_datasrc, io)
io.close()
print(ofile)

In [ ]:
dir(io)

# That was Graphcast. Now try Pangu

In [ ]:
import plot_ensemble
from earyth2studio.models import Pangu6
from metpy.constants import Re
from s3_run_pangu_ecmwf import run_inference, setup_model_sessions

# date = pd.to_datetime("2024042400", format="%Y%m%d%H")
ic = "mpas"
fhr_end = 120

In [ ]:
ort_session_24, ort_session_6 = setup_model_sessions(ai_models_dir)

In [ ]:
output_dir = Path(f"/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather/{date:%Y%m%d%H}")

da = ds.rename(variable="channel")
inferences = run_inference(da, ort_session_24, ort_session_6, fhr_end)
for fcst in inferences:
    fhr = fcst.prediction_timedelta.squeeze() / pd.to_timedelta("1h")
    output_filename = os.path.join(output_dir, f"pangu_{ic}_hybrid_pred_{fhr:03.0f}.nc")
    print(output_filename)
    # zlib compression can reduce by 1/3 but takes long time (versus almost instantaneous)
    fcst.to_netcdf(output_filename)

In [ ]:
output_dir = Path(f"/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather/{date:%Y%m%d%H}")

for member in ds.member.data:
    da = ds.sel(member=member).rename(variable="channel").rename(Time="time")
    all_exist = True
    for fhr in range(6, fhr_end + 1, 6):
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        if not os.path.exists(output_filename):
            print(output_filename, "no exists")
            all_exist = False
            break
    if all_exist:
        print(f"all {date} {ic}{member} exist")
        continue
    inferences = run_inference(da, ort_session_24, ort_session_6, fhr_end)
    for fcst in inferences:
        fhr = fcst.prediction_timedelta.squeeze() / pd.to_timedelta("1h")
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        print(output_filename)
        # zlib compression can reduce by 1/3 but takes long time (versus almost instantaneous)
        fcst.to_netcdf(output_filename)

In [ ]:
ifiles = []
for member in ds.member.data:
    ifiles.append(sorted(list(output_dir.glob(f"pangu_{ic}{member}_pred_???.nc"))))
da = (
    xr.open_mfdataset(
        ifiles,
        combine="nested",
        concat_dim=["member", "prediction_timedelta"],
    )
    .rename(lat="latitude", lon="longitude")
    .assign_coords(member=ds.member)
    .sel(channel="z500")
    .rename(__xarray_dataarray_variable__="z", prediction_timedelta="step")
    .squeeze(dim="init_time")
)
da

In [ ]:
import argparse

args = argparse.Namespace
args.ic = ic.upper()
args.model = "panguweather"
fig = plot_ensemble.plot_forecast_grid(
    args, da.sortby("member"), plotdays=range(1, fhr_end // 24 + 1)
)

In [ ]:
member = 4
inferences = xr.open_mfdataset(output_dir.glob(f"pangu_{ic}{member}_pred_???.nc"))
inferences = inferences.sel(lat=slice(60, 20), lon=slice(220, 300))
inferences.sel(channel="z500").squeeze().__xarray_dataarray_variable__.plot(
    col="prediction_timedelta", col_wrap=6
)

In [ ]:
inferences